In [4]:
pip install cupy-cuda12x  



  Using cached cupy_cuda12x-13.3.0-cp312-cp312-win_amd64.whl.metadata (2.7 kB)
  Using cached fastrlock-0.8.2-cp312-cp312-win_amd64.whl.metadata (9.6 kB)
Using cached cupy_cuda12x-13.3.0-cp312-cp312-win_amd64.whl (69.5 MB)
Using cached fastrlock-0.8.2-cp312-cp312-win_amd64.whl (30 kB)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cupy as cp
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

data = pd.read_csv('credit_card_fraud_dataset.csv')


X = data.drop('IsFraud', axis=1)  # Features
y = data['IsFraud']  # Labels

data['TransactionDate'] = pd.to_datetime(data['TransactionDate'])
data['TransactionHour'] = data['TransactionDate'].dt.hour
data['TransactionDayOfWeek'] = data['TransactionDate'].dt.dayofweek
data = data.drop('TransactionDate', axis=1)

from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in ['TransactionType', 'Location']:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

X = data.drop('IsFraud', axis=1)
y = data['IsFraud']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_cp = cp.asarray(X_scaled)  

X_train_cp, X_test_cp, y_train_cp, y_test_cp = train_test_split(X_scaled_cp, y, test_size=0.2, random_state=42)

X_train = cp.asnumpy(X_train_cp)
X_test = cp.asnumpy(X_test_cp)
y_train = cp.asnumpy(y_train_cp)
y_test = cp.asnumpy(y_test_cp)

svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19787
           1       0.00      0.00      0.00       213

    accuracy                           0.99     20000
   macro avg       0.49      0.50      0.50     20000
weighted avg       0.98      0.99      0.98     20000



c:\Users\SHAURYA MATHUR\Desktop\Programming\ML\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\SHAURYA MATHUR\Desktop\Programming\ML\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\SHAURYA MATHUR\Desktop\Programming\ML\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [11]:
################################## RANDOM INPUT VALUE FOR DETECTION ######################################

sample = {
    'TransactionID': 482,
    'Amount': 250000.00,
    'MerchantID': 712,
    'TransactionType': 'refund',  
    'Location': 'New York',  
    'TransactionHour': 14,
    'TransactionDayOfWeek': 6
}


sample['TransactionType'] = label_encoders['TransactionType'].transform([sample['TransactionType']])[0]
sample['Location'] = label_encoders['Location'].transform([sample['Location']])[0]

sample_array = np.array([[sample['TransactionID'], sample['Amount'], sample['MerchantID'], 
                          sample['TransactionType'], sample['Location'], 
                          sample['TransactionHour'], sample['TransactionDayOfWeek']]])

sample_scaled = scaler.transform(sample_array)

prediction = svm_model.predict(sample_scaled)
print("Predicted Fraud Status (0 = No Fraud, 1 = Fraud):", prediction[0])


Predicted Fraud Status (0 = No Fraud, 1 = Fraud): 0


c:\Users\SHAURYA MATHUR\Desktop\Programming\ML\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [12]:
import pickle

# Assuming `svm_model`, `scaler`, and `label_encoders` are already trained in previous cells

# Save the SVM model
with open('svm_model.pkl', 'wb') as model_file:
    pickle.dump(svm_model, model_file)

# Save the scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

# Save the label encoders
with open('label_encoders.pkl', 'wb') as encoder_file:
    pickle.dump(label_encoders, encoder_file)

print("Model, scaler, and label encoders have been saved successfully.")


Model, scaler, and label encoders have been saved successfully.
